In [14]:
# Importação das bibliotecas

import pandas as pd
import numpy as np

In [15]:
# Importação do arquivo CSV dos dados

arquivo = 'C:/Users/******/Desktop/********/Análise do Comportamento de Compra - E-commerce/Vendas_Final.csv'
df_vendas = pd.read_csv(arquivo, sep = ';')
df_vendas.head() # Exibição do cabeçalho do dataframe

,Unnamed: 0,GENERO,ID_CLIENTE,ID_PRODUTO,VALOR_UNITARIO,QUANTIDADE,VALOR_TOTAL,IDADE
0,16939,Masculino,380,120030001,49.68,3,149.04,74
1,16940,Masculino,380,120070003,35.28,3,105.84,74
2,16941,Masculino,380,850040007,207.00,1,207.00,74
3,10685,Masculino,806,850040007,207.00,1,207.00,56
4,10682,Masculino,806,E930030026,23.80,2,47.60,56


In [16]:
# Seleção das colunas do dataframe que serão utilizadas e exibição do resultado

df_vendas = df_vendas.iloc[:, 1:8]
df_vendas.head()

,GENERO,ID_CLIENTE,ID_PRODUTO,VALOR_UNITARIO,QUANTIDADE,VALOR_TOTAL,IDADE
0,Masculino,380,120030001,49.68,3,149.04,74
1,Masculino,380,120070003,35.28,3,105.84,74
2,Masculino,380,850040007,207.00,1,207.00,74
3,Masculino,806,850040007,207.00,1,207.00,56
4,Masculino,806,E930030026,23.80,2,47.60,56


## Informações sobre os clientes

In [17]:
# Seleção das colunas do dataframe que serão utilizadas e exibição do resultado

clientes_demografia = df_vendas.loc[:, ["GENERO", "ID_CLIENTE", "IDADE"]]
clientes_demografia.head()

,GENERO,ID_CLIENTE,IDADE
0,Masculino,380,74
1,Masculino,380,74
2,Masculino,380,74
3,Masculino,806,56
4,Masculino,806,56


In [18]:
# Remoção de duplicados e contador de número de clientes

clientes_demografia = clientes_demografia.drop_duplicates()
clientes_contador = clientes_demografia.count()[0]
clientes_contador # Exibição dos dados

2616

In [19]:
# Converte o resultado para um dataframe para uso posterior na análise

pd.DataFrame({"Total de clientes" : [clientes_contador]})

,Total de clientes
0,2616


## Análise de compra

In [20]:
# Cálculo iniciais

item_id = len(df_vendas["ID_PRODUTO"].unique())
num_compras_item = df_vendas["QUANTIDADE"].sum()
total_vendas_item = df_vendas["VALOR_TOTAL"].sum()
preco_medio_item = df_vendas["VALOR_UNITARIO"].mean()

# Exporta os resultados para um dataframe

resumo_calculos = pd.DataFrame({"Número de itens únicos" : item_id, 
                                "Quantidade de itens comprados" : num_compras_item, 
                                "Valor total de vendas" : total_vendas_item, 
                                "Preço médio" : [preco_medio_item]})

resumo_calculos = resumo_calculos.round(2) # Ajusta os números para 2 casas decimais
resumo_calculos["Valor total de vendas"] = resumo_calculos["Valor total de vendas"].map("R${:,.2f}".format) # Adiciona o R$ no Valor total de vendas
resumo_calculos["Preço médio"] = resumo_calculos["Preço médio"].map("R${:,.2f}".format) # Adiciona o R$ no preço médio
resumo_calculos = resumo_calculos.loc[:, ["Número de itens únicos", "Preço médio", "Quantidade de itens comprados",
                                          "Valor total de vendas"]] # Ordena as colunas

resumo_calculos # Exibição dos dados

,Número de itens únicos,Preço médio,Quantidade de itens comprados,Valor total de vendas
0,259,R$49.59,51477,"R$1,894,547.64"


## Informações demográficas

In [21]:
# Cálculos iniciais

genero_contador = clientes_demografia["GENERO"].value_counts()
genero_percentual = (genero_contador / clientes_contador) * 100

# Exportação dos resultados para um dataframe

genero_demografia = pd.DataFrame({"GENERO" : genero_contador, 
                                  "%" : genero_percentual})

genero_demografia = genero_demografia.round(2) # Ajusta os números para 2 casas decimais
genero_demografia["%"] = genero_demografia["%"].map("{:,.1f}%".format) # Cria uma coluna no dataframe com o percentual

genero_demografia # Exibição dos dados

,GENERO,%
Masculino,1826,69.8%
Feminino,790,30.2%


## Análise de compra por gênero

In [22]:
# Agrupamentos

genero_num_compras_item = df_vendas.groupby(["GENERO"]).sum()["QUANTIDADE"].rename("Quantidade de itens comprados")
genero_preco_medio_item = df_vendas.groupby(["GENERO"]).mean()["VALOR_UNITARIO"].rename("Preço médio")
genero_total_vendas_item = df_vendas.groupby(["GENERO"]).sum()["VALOR_TOTAL"].rename("Valor total de vendas")

# Exportação dos resultados para um dataframe

genero_analise_compra = pd.DataFrame({"Quantidade de itens comprados" : genero_num_compras_item, 
                                      "Preço médio" : genero_preco_medio_item, 
                                      "Valor total de vendas" : genero_total_vendas_item})

genero_analise_compra = genero_analise_compra.round(2) # Ajusta os números para 2 casas decimais
genero_analise_compra["Preço médio"] = genero_analise_compra["Preço médio"].map("R${:,.2f}".format) # Adiciona o R$ no preço médio
genero_analise_compra["Valor total de vendas"] = genero_analise_compra["Valor total de vendas"].map("R${:,.2f}".format) # Adiciona o R$ no Valor total de vendas

genero_analise_compra # Exibição dos dados

,Quantidade de itens comprados,Preço médio,Valor total de vendas
GENERO,,,
Feminino,12280,R$46.90,"R$439,737.34"
Masculino,39197,R$50.51,"R$1,454,810.30"


## Análise demográfica

In [23]:
# Cria uma nova coluna nos dataframes definindo o grupo de intervalo de idade que o cliente pertence

idade_bins = [0, 14.99, 24.99, 34.99, 44.99, 54.99, 64.99, 74.99, 84.99, 999]
idade_agrupamentos = ["Menos de 15", "15 a 24", "25 a 34", "35 a 44", "45 a 54", "55 a 64", "65 a 74", "75 a 84", "Mais de 84"]
df_vendas["Intervalo de idades"] = pd.cut(df_vendas["IDADE"], idade_bins, labels = idade_agrupamentos)
clientes_demografia["Intervalo de idades"] = pd.cut(clientes_demografia["IDADE"], idade_bins, labels = idade_agrupamentos)

# Cálculos para execução da análise

idade_demografia_contador = clientes_demografia["Intervalo de idades"].value_counts()
idade_demografia_percentual = (idade_demografia_contador / clientes_contador) * 100
idade_demografia_preco_medio_item = df_vendas.groupby(["Intervalo de idades"]).mean()["VALOR_UNITARIO"]
idade_demografia_total_vendas_item = df_vendas.groupby(["Intervalo de idades"]).sum()["VALOR_TOTAL"]

# Exportação dos resultados para um dataframe

idade_demografia = pd.DataFrame({"N° Clientes": idade_demografia_contador, "%": idade_demografia_percentual, 
                                 "Preço médio": idade_demografia_preco_medio_item,
                                 "Valor total de vendas": idade_demografia_total_vendas_item})

idade_demografia["%"] = idade_demografia["%"].map("{:,.2f}%".format) # Adiciona o % na coluna %
idade_demografia["Preço médio"] = idade_demografia["Preço médio"].map("R${:,.2f}".format) # Adiciona o R$ no preço médio
idade_demografia["Valor total de vendas"] = idade_demografia["Valor total de vendas"].map("R${:,.2f}".format) # Adiciona o R$ no valor total de vendas

idade_demografia = idade_demografia.sort_index() # Ordenação pelo índice
idade_demografia # Exibição dos dados

,N° Clientes,%,Preço médio,Valor total de vendas
Menos de 15,1,0.04%,R$275.90,R$275.90
15 a 24,26,0.99%,R$54.98,"R$8,911.40"
25 a 34,258,9.86%,R$44.19,"R$118,422.91"
35 a 44,691,26.41%,R$49.34,"R$393,191.90"
45 a 54,673,25.73%,R$50.88,"R$534,632.42"
55 a 64,598,22.86%,R$47.87,"R$530,327.86"
65 a 74,293,11.20%,R$49.62,"R$229,428.18"
75 a 84,70,2.68%,R$67.38,"R$74,976.67"
Mais de 84,6,0.23%,R$57.62,"R$4,380.40"


## Maiores compradores

In [24]:
# Agrupamentos

id_cliente_total = df_vendas.groupby(["ID_CLIENTE"]).sum()["VALOR_TOTAL"].rename("Valor total de compras")
id_cliente_media = df_vendas.groupby(["ID_CLIENTE"]).mean()["VALOR_UNITARIO"].rename("Valor médio de compra")
id_cliente_contador = df_vendas.groupby(["ID_CLIENTE"]).sum()["QUANTIDADE"].rename("Total de itens comprados")

# Exportação dos resultados para um dataframe

df_id_cliente = pd.DataFrame({"Valor total de compras": id_cliente_total, "Valor médio de compra": id_cliente_media, 
                              "Total de itens comprados": id_cliente_contador})

df_id_cliente["Valor total de compras"] = df_id_cliente["Valor total de compras"].map("R${:,.2f}".format) # Adiciona o R$ no valor total de compras
df_id_cliente["Valor médio de compra"] = df_id_cliente["Valor médio de compra"].map("R${:,.2f}".format) # Adiciona o R$ no valor médio de compra
df_id_cliente.sort_values("Total de itens comprados", ascending = False).head(5) # Ordena os registros pelo total de itens comprados do maior para o menor

,Valor total de compras,Valor médio de compra,Total de itens comprados
ID_CLIENTE,,,
17368,"R$12,711.34",R$38.11,404
24334,"R$12,300.30",R$38.30,326
21104,"R$10,417.85",R$35.59,307
21711,"R$5,176.40",R$30.35,301
21114,"R$7,509.20",R$23.05,294


## Itens mais comprados

In [25]:
# Agrupamentos

itens_total = df_vendas.groupby(["ID_PRODUTO"]).sum()["VALOR_TOTAL"].rename("Valor total de compras")
itens_media = df_vendas.groupby(["ID_PRODUTO"]).mean()["VALOR_UNITARIO"].rename("Valor médio de compra")
itens_contador = df_vendas.groupby(["ID_PRODUTO"]).sum()["QUANTIDADE"].rename("Total de itens comprados")

# Exportação dos resultados para um dataframe

df_itens = pd.DataFrame({"Valor total de compras": itens_total, "Valor médio de compra": itens_media, 
                         "Total de itens comprados": itens_contador})

df_itens["Valor total de compras"] = df_itens["Valor total de compras"].map("R${:,.2f}".format) # Adiciona o R$ no valor total de compras
df_itens["Valor médio de compra"] = df_itens["Valor médio de compra"].map("R${:,.2f}".format) # Adiciona o R$ no valor médio de compra
df_itens.sort_values("Total de itens comprados", ascending = False).head(5) # Ordena os registros pelo total de itens comprados do maior para o menor

,Valor total de compras,Valor médio de compra,Total de itens comprados
ID_PRODUTO,,,
E930030026,"R$95,813.94",R$21.45,4631
590020001,"R$14,098.60",R$3.90,3919
E930150002,"R$53,120.70",R$20.22,2662
E930060016,"R$53,119.47",R$26.40,2050
E930030030,"R$30,295.59",R$17.23,1745


## Itens mais lucrativos

In [30]:
# Agrupamentos

itens2_total = df_vendas.groupby(["ID_PRODUTO"]).sum()["VALOR_TOTAL"].rename("Valor total de compras")
itens2_media = df_vendas.groupby(["ID_PRODUTO"]).mean()["VALOR_UNITARIO"].rename("Valor médio de compra")
itens2_contador = df_vendas.groupby(["ID_PRODUTO"]).sum()["QUANTIDADE"].rename("Total de itens comprados")

# Exportação dos resultados para um dataframe

df_itens2 = pd.DataFrame({"Valor total de compras": itens2_total, "Valor médio de compra": itens2_media, 
                         "Total de itens comprados": itens2_contador})

df_itens2 = df_itens2.sort_values("Valor total de compras", ascending = False).head(5) # Ordena os registros pelo valor total de compras do maior para o menor

df_itens2["Valor total de compras"] = df_itens2["Valor total de compras"].map("R${:,.2f}".format) # Adiciona o R$ no valor total de compras
df_itens2["Valor médio de compra"] = df_itens2["Valor médio de compra"].map("R${:,.2f}".format) # Adiciona o R$ no valor médio de compra

df_itens2.head(5) # Exibe o top 5

,Valor total de compras,Valor médio de compra,Total de itens comprados
ID_PRODUTO,,,
850040007,"R$97,030.48",R$243.45,399
E930030026,"R$95,813.94",R$21.45,4631
E930030029,"R$93,947.42",R$119.87,767
120050001,"R$77,792.85",R$67.40,1240
870020003,"R$63,645.40",R$150.46,425
